In [1]:
from pynq import Overlay
from pynq.lib.video import *
from pynq import allocate

In [2]:
import cv2
import numpy as np
import random
from PIL import Image
import time

In [3]:
base = Overlay("design_1.bit")

In [4]:
base.ip_dict

{'arithm_accel_0/s_axi_control': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb4453db0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'arithm_accel_0/s_axi_control',
  'gpio': {},
  'interrupts': {},
  'mem_id': 's_axi_control',
  'parameters': {'C_M_AXI_GMEM0_ADDR_WIDTH': '64',
   'C_M_AXI_GMEM0_ARUSER_WIDTH': '1',
   'C_M_AXI_GMEM0_AWUSER_WIDTH': '1',
   'C_M_AXI_GMEM0_BUSER_WIDTH': '1',
   'C_M_AXI_GMEM0_CACHE_VALUE': '"0011"',
   'C_M_AXI_GMEM0_DATA_WIDTH': '32',
   'C_M_AXI_GMEM0_ENABLE_ID_PORTS': 'false',
   'C_M_AXI_GMEM0_ENABLE_USER_PORTS': 'false',
   'C_M_AXI_GMEM0_ID_WIDTH': '1',
   'C_M_AXI_GMEM0_PROT_VALUE': '"000"',
   'C_M_AXI_GMEM0_RUSER_WIDTH': '1',
   'C_M_AXI_GMEM0_USER_VALUE': '0x00000000',
   'C_M_AXI_GMEM0_WUSER_WIDTH': '1',
   'C_M_AXI_GMEM1_ADDR_WIDTH': '64',
   'C_M_AXI_GMEM1_ARUSER_WIDTH': '1',
   'C_M_AXI_GMEM1_AWUSER_WIDTH': '1',
   'C_M_AXI_GMEM1_BUSER_WIDTH': '1',
   'C_M_AXI_GMEM1_CACHE_VALUE': '"0011"',
   'C_M_AXI_GMEM1_

In [5]:
dma0 = base.gesture.axi_dma_0
dma1 = base.gesture.axi_dma_1
cnn_ip = base.gesture.cnn_top_0
ip_resize = base.resize_accel_0
ip_arithm = base.arithm_accel_0
ip_thres = base.threshold_accel_0
ip_dilation = base.dilation_accel_0

In [6]:
gesture_list = ["O", "S", "V", "W"]

## HDMI Start

In [7]:
horizontal = 640
vertical = 480
color = 24

In [8]:
Mode = VideoMode(horizontal, vertical, color)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode, PIXEL_BGR) #PIXEL_BGR

In [9]:
hdmi_out.start()

## FC

In [10]:
# FC 0
dense_1_kernel  = np.load('./fc_weights/dense_kernel_0.npy') ### 16
dense_1_kernel  = dense_1_kernel.transpose((1,0))
dense_1_bias    = np.load('./fc_weights/dense_bias_0.npy')
# FC 1
dense_2_kernel  = np.load('./fc_weights/dense_1_kernel_0.npy') ### 16
dense_2_kernel  = dense_2_kernel.transpose((1,0))
dense_2_bias    = np.load('./fc_weights/dense_1_bias_0.npy')
# FC 2
dense_3_kernel  = np.load('./fc_weights/dense_2_kernel_0.npy') ### 16
dense_3_kernel  = dense_3_kernel.transpose((1,0))
dense_3_bias    = np.load('./fc_weights/dense_2_bias_0.npy')
# FC 3
dense_4_kernel  = np.load('./fc_weights/dense_3_kernel_0.npy') ### 16
dense_4_kernel  = dense_4_kernel.transpose((1,0))
dense_4_bias    = np.load('./fc_weights/dense_3_bias_0.npy')

## Webcam

In [11]:
# camera (input) configuration
frame_in_w = 640
frame_in_h = 480
frame_in_c = 3

In [12]:
videoIn = cv2.VideoCapture(0)

videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))

capture device is open: True


In [13]:
frame_gray_old = np.zeros((32, 32), 'uint8')

In [14]:
def optimized_Otsu(img):
    threshold = {'value':16}
    histogram = np.zeros((256), np.uint16)
    integral_array = np.zeros((256), np.uint32)
    between_variance = []
    bt_v_i = 0
    u1_array = []
    u2_array = []
    for x in range(0, 32):
        for y in range(0, 32):
            histogram[img[y][x]] += 1
    #print(histogram)       
    #Integral Image        
    for value, num in enumerate(histogram):
        #At least > 5
        if value > 16:
            integral_array[value] = num +  integral_array[value-1]
        else:
            histogram[value] = 0

    for i, value in enumerate(integral_array):
        if (value > integral_array[255]/2 and threshold['value'] == 16):
            threshold['value'] = i
    return between_variance, threshold, integral_array, histogram
    #     return

## Data

In [15]:
from pynq import allocate
path = './cnn_weights.txt'
f = open(path, 'r')
input_buffer = allocate(shape=(170,), dtype=np.int32)
for i in range(170):
    line=f.readline()
    input_buffer[i]=(int(line,base=10))

print(input_buffer)

[ 1221  2634  5117  4319  5415  6387  4371  3708  2700 -1508    35  3606
     6  -495  3006 -5757  1488  1219 -1502   735  -678   118 -3892  2714
    14 -2310  2940  -629  1806  1709   828 -3714 -5127 -1543 -6152  -942
  2580 -3719  2163   927  1147  3782  1456 -1382  2384   835  1148  1095
  3375    -2   242  2409  3775 -1079  4071  3036  1505  2791   768  2983
  1363   771  -336  2205  1431  1167  2318  1789  -993 -3204   883   784
  2438 -1238  1143  3251  2988   105   470  -233   359   821  1032  -120
 -3297  1562 -2064 -1441   630   -74   936  -648  2370  3163   596 -1921
 -4966  -894  -534   508   220  1788 -1450  3238  2183  2356   918  1962
  1775 -2222   -46  1091   938  2220  1435  2164  -265    -7 -3175  -243
   361   911 -1263   241 -4298  1347   763  2714  1631   855   498  1757
   518  -473  1265  3629   494  -397  1545  2101  1811  -788    66  1175
 -1065 -5853 -1413  4192  3794  1390   562  -856  1154  -944  2690   303
 -1734  4320  -493     0   133   -13  2704  4030   

In [16]:
input_data_buffer = allocate(shape=(1024,), dtype=np.int32)

In [17]:
output_buffer = allocate(shape=(65,), dtype=np.int32)

In [18]:
resize_inBuffer  = allocate(shape=(640*480,), dtype=np.uint8)
resize_outBuffer = allocate(shape=(32*32,), dtype=np.uint8)

arithm_outBuffer = allocate(shape=(32*32,), dtype=np.uint8)
diff_buffer = allocate(shape=(32*32,), dtype=np.uint8)

threshold_outBuffer = allocate(shape=(32*32,), dtype=np.uint8)

dilation_outBuffer = allocate(shape=(32*32,), dtype=np.uint8)
kerBuffer          = allocate(shape=(9,), dtype=np.uint8)

In [19]:
input_kernel = [0, 1, 0, 1, 1, 1, 0, 1, 0]
for i in range(9):
    kerBuffer[i] = input_kernel[i]

## initial

In [22]:
timer = 0
frame = np.zeros((frame_in_h, frame_in_w, frame_in_c), 'uint8')
frame_dilate = np.zeros((32, 32), 'uint8')
frame_bin    = np.zeros((32, 32), 'uint8')
frame_out = np.zeros((240, 240), 'uint8')
frame_line = np.zeros((32, 32, 3), 'uint8')
frame_draw = np.zeros((240, 240, 3), 'uint8')
canvas = np.zeros((32, 32, 3), 'uint8')

pre_x = 0
pre_y = 0
timer = 0
count = 0
x = 0
y = 0
result = ''
flatten = np.zeros((64),dtype = np.float32)
time_start = 0
time_end = 0
fps = 0
keep_timer = 30
coor_cnt = 0

## Main

In [23]:
cnn_ip.write(0x00,0x01)
    
while True:
    
    ret, frame_vga = videoIn.read()
    
    ##################################
    #         Flip images            #
    ##################################
    frame_vga = cv2.flip(frame_vga, 1)
    
    if(keep_timer == 0):
        ##################################
        #             Gray               #
        ##################################
        frame_gray = cv2.cvtColor(frame_vga, cv2.COLOR_BGR2GRAY)
        
        ##################################
        #             Resize             #
        ##################################
        frame_gray = cv2.resize(frame_gray, (32, 32))
        
#         for i in range(640):
#             for j in range(480):
#                 resize_inBuffer[j*640+i]=frame_gray[j][i]
                
#         ip_resize.write(0x10, resize_inBuffer.device_address)
#         ip_resize.write(0x1C, resize_outBuffer.device_address)
#         ip_resize.write(0x28, 64)
#         ip_resize.write(0x30, 64)
#         ip_resize.write(0x38, 32)
#         ip_resize.write(0x40, 32)
        
#         ip_resize.write(0x00, 0x01)
        
        ##################################
        #           Difference           #
        ##################################
        frame_absdiff = cv2.absdiff(frame_gray, frame_gray_old)
        
#         for i in range(1024):
#             diff_buffer[i] = frame_gray_old[i%32][i//32]
    
#         ip_arithm.s_axi_control_r.write(0x10, resize_outBuffer.device_address)
#         ip_arithm.s_axi_control_r.write(0x1C, diff_buffer.device_address)
#         ip_arithm.s_axi_control_r.write(0x28, arithm_outBuffer.device_address)
        
#         ip_arithm.s_axi_control.write(0x00, 0x01)
        
#         for i in range(1024):
#             frame_gray_old[i%32][i//32] = arithm_outBuffer[i]

        frame_gray_old = frame_gray

        ##################################
        #         Binarization           #
        ##################################
        
#         frame_blur = cv2.medianBlur(frame_absdiff, 3)
#         ret, frame_bin = cv2.threshold(frame_blur, 40 ,255, cv2.THRESH_BINARY)
        
        for i in range(1024):
            arithm_outBuffer[i] = frame_absdiff[i%32][i//32]
    
        ip_thres.s_axi_control_r.write(0x10, arithm_outBuffer.device_address)
        ip_thres.s_axi_control_r.write(0x1c, threshold_outBuffer.device_address)
        
        ip_thres.s_axi_control.write(0x10, 40)  #threshold
        ip_thres.s_axi_control.write(0x18, 255) #MaxVal
        ip_thres.s_axi_control.write(0x20, 32)  #cols
        ip_thres.s_axi_control.write(0x28, 32)  #rows

        ip_thres.s_axi_control.write(0x00, 0x01)
        
        for i in range(1024):
            frame_bin[i%32][i//32] = threshold_outBuffer[i]
            
    #     _,th,_,_ = optimized_Otsu(frame_absdiff)
    #     ret, frame_otsu = cv2.threshold(src=frame_absdiff, thresh=th['value'], maxval=255, type=cv2.THRESH_BINARY)

        ##################################
        #             Dilate             #
        ##################################
        
        kernel_1 = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
        frame_dilate = cv2.dilate(frame_bin, kernel_1, iterations = 1)

#         ip_dilation.write(0x10, threshold_outBuffer.device_address)
#         ip_dilation.write(0x1C, dilation_outBuffer.device_address)
#         ip_dilation.write(0x28, kerBuffer.device_address)

#         ip_dilation.write(0x00, 0x01)
        
#         for i in range(1024):
#             frame_dilate[i%32][i//32] = dilation_outBuffer[i]
        
        ##################################
        #    Max connected component     #
        ##################################
        nb_components, labels, stats, centroids = cv2.connectedComponentsWithStats(frame_dilate, connectivity=4)
        sizes = stats[:, -1] # area
        if(nb_components > 1):
            ret = True
            x = stats[sorted(range(len(sizes)), key=sizes.__getitem__)[-2]][0]
            y = stats[sorted(range(len(sizes)), key=sizes.__getitem__)[-2]][1]
        else: 
            ret = False

        ##################################
        #            Draw Line           #
        ##################################
        if(ret):
            print(coor_cnt)
            if(coor_cnt > 1): 
                frame_line = cv2.line(canvas, (pre_x, pre_y), (x, y), (0, 255, 255), 1)
                print('draw')
            coor_cnt = coor_cnt + 1
            pre_x = x
            pre_y = y
    
    ##################################
    #            Show Frame          #
    ##################################
    
    frame_draw = cv2.max(cv2.resize(frame_vga, (480, 480)),cv2.resize(frame_line, (480, 480)))
    
    frame_word = np.vstack([np.zeros((160, 160, 3), 'uint8'),
                         cv2.resize(np.stack((frame_out,)*3, axis=-1), (160, 160)),
                         np.zeros((160, 160, 3), 'uint8')])
    
    frame = np.hstack([frame_draw, frame_word])
    
    cv2.rectangle(frame,(480, 0), (638, 160), (255, 255, 255), 1)
    
    if(keep_timer == 0):
        cv2.circle(frame, (500, 22), 4, (0, 0, 255), 8)
        cv2.putText(img=frame,
                    text='REC',
                    org=(512, 30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.7,
                    color=(0, 0, 255), # b g r
                    thickness=2,
                    lineType=cv2.LINE_AA)
            
        cv2.putText(img=frame,
                    text=str((50-timer)//10), 
                    org=(520, 140),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=4,
                    color=(0, 0, 255),
                    thickness=2,
                    lineType=cv2.LINE_AA)
    else:
        cv2.putText(img=frame,
                    text='Ready:',
                    org=(490, 30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.5,
                    color=(255, 255, 255), # b g r
                    thickness=1,
                    lineType=cv2.LINE_AA)
        
        cv2.putText(img=frame,
                    text=str(keep_timer//10), 
                    org=(520, 140),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=4,
                    color=(255, 255, 255),
                    thickness=1,
                    lineType=cv2.LINE_AA)
        
    
    cv2.rectangle(frame,(480, 160), (638, 320), (255, 255, 255), 1)
    
    cv2.rectangle(frame,(480, 320), (638, 478), (255, 255, 255), 1)
    cv2.putText(img=frame,
                text="Result: ", 
                org=(490, 350),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.6,
                color=(255, 255, 255),
                thickness=1,
                lineType=cv2.LINE_AA)
    
    cv2.putText(img=frame,
                text=result, 
                org=(520, 460),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=4,
                color=(255, 255, 255),
                thickness=2,
                lineType=cv2.LINE_AA)
    
    ##################################
    #             HDMI               #
    ##################################
    outframe = hdmi_out.newframe()
    outframe[:] = frame
    hdmi_out.writeframe(outframe)
   
    ##################################
    #          Gesture Time          #
    ##################################
    
    if(keep_timer == 0):
        timer = timer + 1
    else:
        keep_timer = keep_timer - 1
        
    time_end = time.time()
    fps = 1/(time_end - time_start)
    time_start = time_end
    
    if timer >= 50:
        coor_cnt = 0
        keep_timer = 30
        canvas = np.zeros((32, 32, 3), 'uint8')
        timer = 0
        frame_out = frame_line[:,:,2]
        frame_line =np.zeros((32, 32, 3), 'uint8')
        cv2.imwrite('./image.jpg', cv2.resize(frame_out, (32, 32)))
        
        # jpg 2 txt 
        im = Image.open('image.jpg')
        fil = open('inputs.txt', 'w')
        pix = im.load()
        pixel = ' '
        row, column = im.size
        for _y in range(column):
            for _x in range(row):
                pixel = pix[_x, _y]
                fil.write(str(pixel) + '\n')
        fil.close()
        
        # input data
        path = './inputs.txt'
        f = open(path, 'r')
        w=''
        for i in range(1024):
            line=f.readline()
            tmp = (int(line,base=10))
            if tmp > 128:
                input_data_buffer[i] = 1
            else:
                input_data_buffer[i] = 0
        
        # DMA
        
        dma0.sendchannel.transfer(input_data_buffer) #transfer data
        dma1.sendchannel.transfer(input_buffer)      #transfer weight
        dma0.recvchannel.transfer(output_buffer)     #recieve result
        dma0.sendchannel.wait()
        dma1.sendchannel.wait()
        dma0.recvchannel.wait()
        
        # FC
        inputs = np.array(output_buffer[1:65])
        for i in range(4):
            for j in range(16):
                flatten[i*16+j] = inputs[j*4+i]/100000000

        fc_output_1 = dense_1_kernel.dot(flatten)+dense_1_bias
        fc_output_1 = np.maximum(fc_output_1, 0) # ReLU
        
        fc_output_2 = dense_2_kernel.dot(fc_output_1)+dense_2_bias
        fc_output_2 = np.maximum(fc_output_2, 0) # ReLU

        fc_output_3 = dense_3_kernel.dot(fc_output_2)+dense_3_bias
        fc_output_3 = np.maximum(fc_output_3, 0) # ReLU

        fc_output_4 = dense_4_kernel.dot(fc_output_3)+dense_4_bias
        
        result = gesture_list[np.argmax(fc_output_4)]
        print('result = ',result)
    
    ##################################
    #            Shut Down           #
    ##################################
    if(base.btns_gpio.read()):
         break

0
1
2
draw
3
draw
4
draw
5
draw
6
draw
7
draw
8
draw
9
draw
10
draw
11
draw
12
draw
13
draw
14
draw
15
draw
16
draw
17
draw
18
draw
19
draw
20
draw
21
draw
22
draw
23
draw
24
draw
25
draw
26
draw
27
draw
28
draw
29
draw
30
draw
31
draw
32
draw
33
draw
34
draw
35
draw
36
draw
37
draw
38
draw
39
draw
40
draw
41
draw
42
draw
43
draw
44
draw
45
draw
46
draw
47
draw
48
draw
49
draw
result =  S
0
1
2
draw
3
draw
4
draw
5
draw
6
draw
7
draw
8
draw
9
draw
10
draw
11
draw
12
draw
13
draw
14
draw
15
draw
16
draw
17
draw
18
draw
19
draw
20
draw
21
draw
22
draw
23
draw
24
draw
25
draw
26
draw
27
draw
28
draw
29
draw
30
draw
31
draw
32
draw
33
draw
34
draw
35
draw
36
draw
37
draw
38
draw
39
draw
40
draw
41
draw
42
draw
43
draw
44
draw
45
draw
46
draw
47
draw
48
draw
result =  O
0
1
2
draw
3
draw
4
draw
5
draw
6
draw
7
draw
8
draw
9
draw
10
draw
11
draw
12
draw
13
draw
14
draw
15
draw
16
draw
17
draw
18
draw
19
draw
20
draw
21
draw
22
draw
23
draw
24
draw
25
draw
26
draw
27
draw
28
draw
29
draw
30
d

KeyboardInterrupt: 